# **Data Acquisition project**

# Step 1: Kaggle dataset

Downloading Kaggle datasets from : [usgs/earthquake-database](https://www.kaggle.com/datasets/usgs/earthquake-database)

In [1]:
import kagglehub

# link: https://www.kaggle.com/datasets/usgs/earthquake-database


path = kagglehub.dataset_download("usgs/earthquake-database")

print("Path to dataset files:", path)

100%|██████████| 590k/590k [00:00<00:00, 734kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/usgs/earthquake-database/versions/1


### USGS Earthquake Database Description

This dataset contains earthquake records from the USGS (United States Geological Survey) database.
Key features include:
- **Time**: Timestamp of earthquake occurrence
- **Latitude/Longitude**: Geographic coordinates of earthquake epicenter
- **Depth**: Depth of earthquake in kilometers
- **Magnitude**: Earthquake magnitude (typically Richter or moment magnitude scale)
- **Location**: Text description of earthquake location

The dataset allows analysis of seismic patterns, magnitude distributions, and geographic clustering of earthquakes.

In [9]:
import pandas as pd
import numpy as np

df_earthquakes = pd.read_csv(path + '/database.csv')


# Checking content
print("\nHeader :\n")
print(df_earthquakes.head())

# Column informations
print("\nInformations :\n")
print(df_earthquakes.info())


# Missing values
print("\nMissing values :\n")
print(df_earthquakes.isnull().sum())

if len(df_earthquakes) > 10000:
    df_earthquakes = df_earthquakes.sample(n=10000, random_state=42)
    print(f"\nDataset reduced to {len(df_earthquakes)} rows")


Header :

         Date      Time  Latitude  Longitude        Type  Depth  Depth Error  \
0  01/02/1965  13:44:18    19.246    145.616  Earthquake  131.6          NaN   
1  01/04/1965  11:29:49     1.863    127.352  Earthquake   80.0          NaN   
2  01/05/1965  18:05:58   -20.579   -173.972  Earthquake   20.0          NaN   
3  01/08/1965  18:49:43   -59.076    -23.557  Earthquake   15.0          NaN   
4  01/09/1965  13:32:50    11.938    126.427  Earthquake   15.0          NaN   

   Depth Seismic Stations  Magnitude Magnitude Type  ...  \
0                     NaN        6.0             MW  ...   
1                     NaN        5.8             MW  ...   
2                     NaN        6.2             MW  ...   
3                     NaN        5.8             MW  ...   
4                     NaN        5.8             MW  ...   

   Magnitude Seismic Stations  Azimuthal Gap  Horizontal Distance  \
0                         NaN            NaN                  NaN   
1        

# Step 2: Web Scrape a Complementary dataset


# Step 3: Merge

# Step 4: Explore / visualise